In [ ]:
import collections
from octo_spork.expressions import *
from octo_spork.logic import to_cnf, to_dnf
from octo_spork.domain import simplify_intervals, simplify_sets, get_attributes

In [ ]:
x1, x2, x3 = [Attribute('x{}'.format(i+1)) for i in range(3)]
expression = And([
    And([Gt(x1, 1), Lt(x1, 2), Ge(x2, 1)]),
    And([Not(Ge(x1, 1.5)), Ge(x2, 1)])])
print(nice_repr(expression))

In [ ]:
reduced = to_dnf(expression)
print(nice_repr(reduced))

In [ ]:
by_attribute = collections.defaultdict(list)
for clause in reduced.clauses:
    by_attribute[frozenset(get_attributes(clause))].append(clause)

result = to_dnf(And([
    simplify_intervals(And(clausegroup))
    for clausegroup in by_attribute.values()]))

print(nice_repr(result))